# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import copy
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
warnings.filterwarnings('ignore')

data_path = './data/part02/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'], axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
mean_df = df.groupby(['Pclass'])['Fare'].mean().reset_index()
median_df = df.groupby(['Pclass'])['Fare'].median().reset_index()
mode_df = df.groupby(['Pclass'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
max_df = df.groupby(['Pclass'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, median_df, how='left', on=['Pclass'])
temp = pd.merge(temp, mode_df, how='left', on=['Pclass'])
temp = pd.merge(temp, max_df, how='left', on=['Pclass'])
temp.columns = ['Pclass', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']
temp

,Pclass,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,1,84.154687,60.2875,26.55,512.3292
1,2,20.662183,14.2500,13.00,73.5000
2,3,13.675550,8.0500,8.05,69.5500


In [3]:
df = pd.merge(df, temp, how='left', on=['Pclass'])
df = df.drop(['Pclass'], axis=1)
df.head()

,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,13.675550,8.0500,8.05,69.5500
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.154687,60.2875,26.55,512.3292
2,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,13.675550,8.0500,8.05,69.5500
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,84.154687,60.2875,26.55,512.3292
4,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,13.675550,8.0500,8.05,69.5500


In [4]:
# 只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features: {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
MMEncoder = MinMaxScaler()
df = df[num_features]
df = df.fillna(-1)
df.head()

8 Numeric Features: ['Age', 'SibSp', 'Parch', 'Fare', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']



,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,22.0,1,0,7.2500,13.675550,8.0500,8.05,69.5500
1,38.0,1,0,71.2833,84.154687,60.2875,26.55,512.3292
2,26.0,0,0,7.9250,13.675550,8.0500,8.05,69.5500
3,35.0,1,0,53.1000,84.154687,60.2875,26.55,512.3292
4,35.0,0,0,8.0500,13.675550,8.0500,8.05,69.5500


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

答：找出類別型和數值型欄位的關係並做群聚編碼有些改善。

In [5]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_temp = df.drop(['Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max'], axis=1)
train_X = MMEncoder.fit_transform(df_temp)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6408846483794697

In [6]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_temp = df
train_X = MMEncoder.fit_transform(df_temp)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6915102098137199